# Reading HDF5 Files

This notebook describes a little bit of basics about how to read hdf5 files in python using the library/module h5py.

Before we start, we can understand a little about what HDF5 format is. 

From Wikipedia:

**Hierarchical Data Format (HDF) is a set of file formats (HDF4, HDF5) designed to store and organize large amounts of data. Originally developed at the National Center for Supercomputing Applications, it is supported by The HDF Group, a non-profit corporation whose mission is to ensure continued development of HDF5 technologies and the continued accessibility of data stored in HDF.**

HDF5 simplifies the file structure to include only two major types of object:

1. Datasets, which are multidimensional arrays of a homogeneous type.

2. Groups, which are container structures which can hold datasets and other groups.

### General Notes about FIRE HDF5 snapshot files:
The snapshot files from the FIRE simulations are also stored in HDF5 format. Due to their large size, the files are usually broken down into smaller chunks, and a given snapshot may contain around 5-10 individual hdf5 files that should be combined to make a single dataset for a given snapshot. 
The files have some stucture in them. There are 4 different groups that are stored in the FIRE HDF5 files:

1. **Header**: This is the metadata for the given snapshot subfile. It contains a lot of useful information, but mainly contains the number of particles in the snapshot totally and the number of particles in a given smaller individual chunk of the snapshot (when it is broken down into smaller files). This can be used to verify some diagnostics.

2. **PartType0**: This is the gas particle group.
3. **PartType1**: This is the dm particle group.
4. **PartType2**: ....Dummy particles, not really used, as far as I know. 
5. **PartType3**: This is the stellar particle group. 

In [59]:
import h5py
import pprint # this is the pretty print module in python, it prints dictionaries in nicer formats

In [60]:
# we will read an example snapshot to understand how to do this in general
# for any hdf5 file
test_file = h5py.File('../snapdir_440/snapshot_440.0.hdf5', 'r') # load the hdf5 file in read mode

In [61]:
print(test_file)
# as you see, the output says nothing useful. This is because this is an
# HDF5 file that needs some special working.

<HDF5 file "snapshot_440.0.hdf5" (mode r)>


In [62]:
# HDF5 files are organized at a high-level like a python dictionary
# they have keys and values
print(f'Keys in this HDF5 file are: {test_file.keys()}\n')
print(f'Values in this HDF5 file are: {test_file.values()}')

Keys in this HDF5 file are: <KeysViewHDF5 ['Header', 'PartType0', 'PartType1', 'PartType2', 'PartType4']>

Values in this HDF5 file are: ValuesViewHDF5(<HDF5 file "snapshot_440.0.hdf5" (mode r)>)


In [67]:
# as you see above, there is still not a lot of useful information, however, from the list of keys
# we finally see that this file contains 5 different 'datasets'
# lets go ahead and parse the Header dataset from the HDF5 file
# Header in the snapshots is a special type of file that is responsible for
# storing the metadata of the simulation snapshot. It contains very useful information.
header_info = test_file['Header']
pprint.pprint(f'Header Keys are: {header_info.keys()}')
pprint.pprint(f'Header Values are: {header_info.values()}')

'Header Keys are: <KeysViewHDF5 []>'
'Header Values are: ValuesViewHDF5(<HDF5 group "/Header" (0 members)>)'


In [69]:
# since the Header is a special group in the HDF5 file, we need to read it another way
# we need to use .attrs method to look at all keys of the attributes that are defined 
# in the HDF5 Header group
print(f'The attribute keys inside the Header Group is: {header_info.attrs.keys()}')

The attribute keys inside the Header Group is: <KeysViewHDF5 ['BoxSize', 'Flag_Cooling', 'Flag_DoublePrecision', 'Flag_Feedback', 'Flag_IC_Info', 'Flag_Metals', 'Flag_Sfr', 'Flag_StellarAge', 'HubbleParam', 'MassTable', 'NumFilesPerSnapshot', 'NumPart_ThisFile', 'NumPart_Total', 'NumPart_Total_HighWord', 'Omega0', 'OmegaLambda', 'Redshift', 'Time']>


In [70]:
# Now obtain each individual metadata attribute
header_dict = {}
for attribute in header_info.attrs.keys():
    header_dict[attribute] = header_info.attrs[attribute]
pprint.pprint(header_dict)

{'BoxSize': 60000.0,
 'Flag_Cooling': 1,
 'Flag_DoublePrecision': 0,
 'Flag_Feedback': 1,
 'Flag_IC_Info': 3,
 'Flag_Metals': 12,
 'Flag_Sfr': 1,
 'Flag_StellarAge': 1,
 'HubbleParam': 0.702,
 'MassTable': array([0.        , 0.00015806, 0.        , 0.        , 0.        ,
       0.        ]),
 'NumFilesPerSnapshot': 8,
 'NumPart_ThisFile': array([1599950, 2353598,  224541,       0,  209181,       0], dtype=int32),
 'NumPart_Total': array([ 8887776, 10403824,  4845138,        0,  1516048,        0],
      dtype=uint32),
 'NumPart_Total_HighWord': array([0, 0, 0, 0, 0, 0], dtype=uint32),
 'Omega0': 0.272,
 'OmegaLambda': 0.728,
 'Redshift': 0.0,
 'Time': 1.0}


In [76]:
# now we have learned how to read the metadata (the header) from the HDF5 file
# lets demonstrate how to read a given particle type dataset information
gas_particles_group = test_file['PartType0']
# now we can print what all information is stored for gas particles by simply examining 
# the keys of the gas_particles_group
print(f'Gas particles contain the following fields: {gas_particles_group.keys()}')

Gas particles contain the following fields: <KeysViewHDF5 ['ArtificialViscosity', 'Coordinates', 'Density', 'ElectronAbundance', 'InternalEnergy', 'Masses', 'Metallicity', 'NeutralHydrogenAbundance', 'ParticleIDs', 'SmoothingLength', 'StarFormationRate', 'Velocities']>


In [79]:
# now we can do the same as above for dark matter and stellar particles
dm_particles_grp = test_file['PartType1']
star_particles_grp = test_file['PartType4']
print(f'DM particles contain the following fields: {dm_particles_grp.keys()}\n')
print(f'Star particles contain the following fields: {star_particles_grp.keys()}')

DM particles contain the following fields: <KeysViewHDF5 ['Coordinates', 'ParticleIDs', 'Velocities']>

Star particles contain the following fields: <KeysViewHDF5 ['Coordinates', 'Masses', 'Metallicity', 'ParticleIDs', 'StellarFormationTime', 'Velocities']>


In [96]:
# to load the coordinates of gas particles we can do the following now
gas_coordinates = gas_particles_group['Coordinates'][:]
print(gas_coordinates)
print(type(gas_coordinates))
# this is a numpy.ndarray of dimensions number_of_gas_particles_in_this_file*3 (x, y, z coordinates)

[[29523.844 34675.754 25684.854]
 [29520.941 34673.055 25667.387]
 [29501.701 34667.36  25667.828]
 ...
 [32799.797 30466.469 21883.357]
 [32799.047 30468.588 21883.162]
 [32798.85  30468.422 21887.209]]
<class 'numpy.ndarray'>


In [104]:
# similarly we can load other things, if not sure about the dimensions, be sure to check them out individually
# load dark matter particles coordinates
dm_coordinates = dm_particles_grp['Coordinates'][:]
print(dm_coordinates)
print(type(dm_coordinates))
# same as above, it is a number_of_dm_particles_in_this_file*3 (x, y, z coordinates)
# if we want to just load the x-positions of the dm particles, we can also just do 
dm_x_coordinates = dm_particles_grp['Coordinates'][:, 0]
# but when possible try to load and work with ndarrays as they are fast and efficient and the recommended way 
# to do things in numpy. ndarrays allow for vectorization, which is super useful

[[29508.812 34584.164 25751.266]
 [29512.236 34587.633 25697.383]
 [29306.955 34558.953 25545.094]
 ...
 [32799.137 30467.658 21889.268]
 [32798.895 30467.377 21889.062]
 [32799.203 30467.824 21888.955]]
<class 'numpy.ndarray'>


**Similarly you can read any other type of information that is present for an individual group in the HDF5 file.**